# Problem: Which Five Zip Codes Would Be Best for Investment?

This question requires us to define what is meant by "best". Careful consideration brings up two interpretations: first, proven successful investments (think the Google and Amazon of zip codes), and second, early-entry investments that are currently still priced low compared to their potential growth.

First, this project will focus on finding the top five best zip codes for each possibility in one city. Then, the same process will be carried out against the top five best zip codes throughout all the cities in the database. The rational being that the best five zipcodes will be chosen for four investor types: a) investors who prefer solid, proven investments; b) investors who prefer to maximize their gains by investing in properties that have yet to show their greatest gains; c) investors who prefer local investments; and d) investors that prefer to invest in the most profitable properties, regardless of locale.

# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd
import pandas.tseries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np

df = pd.read_csv('zillow_data.csv')
df.head(10)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
5,91733,77084,Houston,TX,Houston,Harris,6,95000.0,95200.0,95400.0,...,157900,158700,160200,161900,162800,162800,162800,162900,163500,164300
6,61807,10467,New York,NY,New York,Bronx,7,152900.0,152700.0,152600.0,...,394400,400000,407300,411600,413200,414300,413900,411400,413200,417900
7,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0,216700.0,216900.0,...,798000,787100,776100,774900,777900,777900,778500,780500,782800,782800
8,91940,77449,Katy,TX,Houston,Harris,9,95400.0,95600.0,95800.0,...,166800,167400,168400,169600,170900,172300,173300,174200,175400,176200
9,97564,94109,San Francisco,CA,San Francisco,San Francisco,10,766000.0,771100.0,776500.0,...,3767700,3763900,3775000,3799700,3793900,3778700,3770800,3763100,3779800,3813500


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


Each of the zip codes is also labeled with the nearest major metropolis. As a resident of Brooklyn, I have chosen to focus on the New York metro area, as I am familiar with it.

In [3]:
nyc_df = df[df['Metro']=='New York']
nyc_df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
6,61807,10467,New York,NY,New York,Bronx,7,152900.0,152700.0,152600.0,...,394400,400000,407300,411600,413200,414300,413900,411400,413200,417900
10,62037,11226,New York,NY,New York,Kings,11,162000.0,162300.0,162600.0,...,860200,851000,853900,870000,885100,887800,890500,901700,930700,963200
12,62087,11375,New York,NY,New York,Queens,13,252400.0,251800.0,251400.0,...,1022600,1033700,1048600,1066400,1081200,1088800,1092700,1089500,1084000,1084600
13,62045,11235,New York,NY,New York,Kings,14,190500.0,191000.0,191500.0,...,767300,777300,788800,793900,796000,799700,806600,810600,813400,816200
20,61625,10011,New York,NY,New York,New York,21,NaN,NaN,NaN,...,12137600,12112600,12036600,12050100,12016300,11946500,11978100,11849300,11563000,11478300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14513,62305,11958,Peconic,NY,New York,Suffolk,14514,177400.0,177200.0,176900.0,...,673600,679300,675900,671400,684900,699900,700100,693200,687200,678400
14530,61831,10518,Lewisboro,NY,New York,Westchester,14531,341700.0,339700.0,337600.0,...,596000,603600,610100,617600,607200,587000,570100,554400,546700,556700
14535,62218,11739,Islip,NY,New York,Suffolk,14536,235000.0,235700.0,236500.0,...,527200,510200,495800,488400,485700,487100,487100,487900,486100,482900
14567,61948,10964,Orangetown,NY,New York,Rockland,14568,261200.0,262400.0,263400.0,...,631700,629800,624000,622300,622400,624300,625100,628000,634100,643000


# Step 2: Data Preprocessing

In [4]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [5]:
get_datetimes(nyc_df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [6]:
nyc_df.isnull().sum()

RegionID      0
RegionName    0
City          0
State         0
Metro         0
             ..
2017-12       0
2018-01       0
2018-02       0
2018-03       0
2018-04       0
Length: 272, dtype: int64

As we can see from the information above, fourteen of the buildings were not constructed in April of 1996. By 2015, all of the buildings were constructed. 

In [7]:
unique, counts = np.unique(nyc_df['RegionName'], return_counts=True)
dict(zip(unique, counts))

{7001: 1,
 7002: 1,
 7003: 1,
 7004: 1,
 7005: 1,
 7006: 1,
 7008: 1,
 7009: 1,
 7010: 1,
 7011: 1,
 7012: 1,
 7013: 1,
 7014: 1,
 7016: 1,
 7020: 1,
 7021: 1,
 7022: 1,
 7023: 1,
 7024: 1,
 7026: 1,
 7027: 1,
 7028: 1,
 7029: 1,
 7030: 1,
 7031: 1,
 7032: 1,
 7033: 1,
 7034: 1,
 7035: 1,
 7036: 1,
 7039: 1,
 7040: 1,
 7041: 1,
 7042: 1,
 7043: 1,
 7044: 1,
 7045: 1,
 7046: 1,
 7047: 1,
 7050: 1,
 7052: 1,
 7054: 1,
 7055: 1,
 7057: 1,
 7058: 1,
 7059: 1,
 7060: 1,
 7062: 1,
 7063: 1,
 7064: 1,
 7065: 1,
 7066: 1,
 7067: 1,
 7068: 1,
 7069: 1,
 7070: 1,
 7071: 1,
 7072: 1,
 7073: 1,
 7074: 1,
 7075: 1,
 7076: 1,
 7077: 1,
 7078: 1,
 7079: 1,
 7080: 1,
 7081: 1,
 7082: 1,
 7083: 1,
 7086: 1,
 7087: 1,
 7088: 1,
 7090: 1,
 7092: 1,
 7093: 1,
 7094: 1,
 7095: 1,
 7103: 1,
 7104: 1,
 7105: 1,
 7106: 1,
 7107: 1,
 7109: 1,
 7110: 1,
 7114: 1,
 7201: 1,
 7202: 1,
 7203: 1,
 7204: 1,
 7205: 1,
 7206: 1,
 7208: 1,
 7302: 1,
 7305: 1,
 7306: 1,
 7307: 1,
 7401: 1,
 7403: 1,
 7405: 1,
 7407: 1,


As we can see from the above list of zipcodes, there is only one property represented per zipcode. This ensures a balanced analysis, as there is an even number of properties across zipcodes. The downside is that a single property may not be a true representation of an entire zipcode.

In [8]:
nyc_df = nyc_df.dropna()

In [9]:
nyc_df['1996-04']

6         152900.0
10        162000.0
12        252400.0
13        190500.0
21       3676700.0
           ...    
14513     177400.0
14530     341700.0
14535     235000.0
14567     261200.0
14620     288000.0
Name: 1996-04, Length: 765, dtype: float64

# Step 3: EDA and Visualization

As we saw above, some of the zip codes were represented by buildings that were not yet constructed in April 1996 (hence the null values). The next step is to find the top five properties with the largest difference between their original value and their respective values in April 2018. 

In [10]:
top_five_df_1996 = []
for i in nyc_df:
    nyc_df_diff = nyc_df['2018-04'] - nyc_df['1996-04']
    top_five_df_1996.append(nyc_df_diff)

pct_diff_1996 = (nyc_df_diff / nyc_df['1996-04'])
pct_diff_1996.nlargest(5)

117     11.189940
1155    10.535523
475      9.942505
191      9.403061
106      8.941958
dtype: float64

In [12]:
top_five_df_1996 = []
for i in nyc_df:
    nyc_df_1996 = (nyc_df['2018-04'] - nyc_df['1996-04'])/nyc_df['1996-04']
    top_five_df_1996.append(nyc_df_1996)

nyc_df_1996.nlargest(5)

117     11.189940
1155    10.535523
475      9.942505
191      9.403061
106      8.941958
dtype: float64

In [11]:
top_five_df_2005 = []
for i in nyc_df:
    nyc_df_diff = nyc_df['2018-04'] - nyc_df['2005-04']
    top_five_df_2005.append(nyc_df_diff)

pct_diff_2005 = (nyc_df_diff / nyc_df['2005-04'])
pct_diff_2005.nlargest(5)

475      2.492899
117      2.372871
1155     2.268325
70       1.663027
13864    1.503284
dtype: float64

In [12]:
top_five_df_2014 = []
for i in nyc_df:
    nyc_df_diff = nyc_df['2018-04'] - nyc_df['2014-04']
    top_five_df_2014.append(nyc_df_diff)

pct_diff_2014 = (nyc_df_diff / nyc_df['2014-04'])
pct_diff_2014.nlargest(5)

660     1.063162
7007    1.012085
475     0.928003
1737    0.922617
190     0.880627
dtype: float64

In [13]:
pct_diff_2014.nlargest(12)

660     1.063162
7007    1.012085
475     0.928003
1737    0.922617
190     0.880627
191     0.832927
1155    0.805223
8495    0.761101
70      0.747542
2043    0.729818
519     0.718695
1065    0.717151
dtype: float64

In [15]:
top_9_pct_diff=nyc_df.loc[[106, 117, 190, 191, 475, 660, 1155, 1737, 7007], '1996-04':'2018-04']
top_9_pct_diff

,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
106,225700.0,227500.0,229400.0,231600.0,234100.0,236900.0,239800.0,242900.0,246400.0,249800.0,...,2173600,2201100,2244400,2266100,2275800,2287100,2288900,2265300,2244900,2243900
117,133200.0,132900.0,132500.0,132200.0,131800.0,131600.0,131500.0,131800.0,132400.0,133100.0,...,1405100,1406400,1424700,1435300,1440500,1463100,1496100,1531100,1581900,1623700
190,124500.0,124500.0,124500.0,124500.0,124700.0,124800.0,125200.0,125500.0,125900.0,126600.0,...,788800,788000,787100,797100,808400,809800,800800,806100,829200,852300
191,137200.0,137800.0,138500.0,139100.0,139600.0,140100.0,140700.0,141400.0,142300.0,143500.0,...,1340200,1372300,1411000,1435900,1446300,1447800,1454900,1453900,1439500,1427300
475,146100.0,146600.0,147200.0,147700.0,148400.0,149300.0,150200.0,151300.0,152700.0,154100.0,...,1452000,1506100,1553100,1567700,1559700,1545700,1540200,1553600,1578400,1598700
660,111400.0,111100.0,110900.0,110600.0,110300.0,110000.0,109700.0,109400.0,109300.0,109400.0,...,465400,469800,475900,484000,495800,506700,510000,515300,531900,545500
1155,149200.0,148400.0,147500.0,146600.0,146000.0,145600.0,145400.0,145600.0,146100.0,146600.0,...,1598000,1623800,1638700,1640400,1644100,1651200,1661800,1672800,1694000,1721100
1737,114300.0,114300.0,114400.0,114500.0,114800.0,115200.0,115700.0,116300.0,117100.0,117900.0,...,439500,449000,458300,468100,478200,487300,496000,510600,531900,546600
7007,230300.0,230800.0,231500.0,232200.0,233100.0,234100.0,235100.0,236100.0,237300.0,238300.0,...,1591400,1600300,1599200,1608000,1632200,1679800,1736600,1749800,1727300,1714900


In [19]:
top_9_pct_diff = top_9_pct_diff.astype(int)
top_9_pct_diff

,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
106,225700,227500,229400,231600,234100,236900,239800,242900,246400,249800,...,2173600,2201100,2244400,2266100,2275800,2287100,2288900,2265300,2244900,2243900
117,133200,132900,132500,132200,131800,131600,131500,131800,132400,133100,...,1405100,1406400,1424700,1435300,1440500,1463100,1496100,1531100,1581900,1623700
190,124500,124500,124500,124500,124700,124800,125200,125500,125900,126600,...,788800,788000,787100,797100,808400,809800,800800,806100,829200,852300
191,137200,137800,138500,139100,139600,140100,140700,141400,142300,143500,...,1340200,1372300,1411000,1435900,1446300,1447800,1454900,1453900,1439500,1427300
475,146100,146600,147200,147700,148400,149300,150200,151300,152700,154100,...,1452000,1506100,1553100,1567700,1559700,1545700,1540200,1553600,1578400,1598700
660,111400,111100,110900,110600,110300,110000,109700,109400,109300,109400,...,465400,469800,475900,484000,495800,506700,510000,515300,531900,545500
1155,149200,148400,147500,146600,146000,145600,145400,145600,146100,146600,...,1598000,1623800,1638700,1640400,1644100,1651200,1661800,1672800,1694000,1721100
1737,114300,114300,114400,114500,114800,115200,115700,116300,117100,117900,...,439500,449000,458300,468100,478200,487300,496000,510600,531900,546600
7007,230300,230800,231500,232200,233100,234100,235100,236100,237300,238300,...,1591400,1600300,1599200,1608000,1632200,1679800,1736600,1749800,1727300,1714900
